In [23]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse

from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

from xgboost import XGBRFRegressor as XGBRF
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.linear_model import LogisticRegression as LogReg

import lazypredict
from lazypredict.Supervised import LazyClassifier as Lc
from lazypredict.Supervised import LazyRegressor as Lr
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR
from sklearn.linear_model import GammaRegressor as GR

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
salaries= pd.read_csv('../data/salaries_data.csv')

test = pd.read_csv('../data/testeo.csv')

In [3]:
#Borramos columnas que no hace falta
test.drop('salary', inplace=True, axis=1)
#test.drop('salary_currency', inplace=True, axis=1)

In [4]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [5]:
def diction(col):
    lista = list(set(list(salaries[col].unique()) + list(test[col].unique())))
    nums =[i for i in range(len(lista))]
    print({(lista[i]): (nums[i]) for i in range(len(lista))})
    return {(lista[i]): (nums[i]) for i in range(len(lista))}

dictio_experience_level = diction('experience_level')
dictio_employment_type = diction('employment_type')
dictio_job_title = diction('job_title')
dictio_employee_residence = diction('employee_residence')
dictio_company_location = diction('company_location')
dictio_company_size = diction('company_size')
dictio_salary_currency = diction('salary_currency')

{'EN': 0, 'SE': 1, 'EX': 2, 'MI': 3}
{'CT': 0, 'FL': 1, 'FT': 2, 'PT': 3}
{'Big Data Engineer': 0, 'Computer Vision Engineer': 1, 'Head of Data': 2, 'Staff Data Scientist': 3, 'Lead Data Scientist': 4, 'Principal Data Scientist': 5, 'Data Analytics Manager': 6, 'Applied Machine Learning Scientist': 7, 'BI Data Analyst': 8, 'Lead Data Analyst': 9, 'Machine Learning Infrastructure Engineer': 10, 'Head of Machine Learning': 11, 'Data Science Manager': 12, 'Machine Learning Scientist': 13, 'Data Engineering Manager': 14, 'Marketing Data Analyst': 15, 'Machine Learning Engineer': 16, 'Financial Data Analyst': 17, 'Data Scientist': 18, 'Lead Machine Learning Engineer': 19, 'ETL Developer': 20, 'Analytics Engineer': 21, 'Head of Data Science': 22, 'Data Science Consultant': 23, 'Lead Data Engineer': 24, 'Director of Data Engineering': 25, 'Principal Data Engineer': 26, 'Principal Data Analyst': 27, 'Applied Data Scientist': 28, 'Product Data Analyst': 29, 'Machine Learning Manager': 30, 'Data

In [6]:
salaries_copy_1 = salaries.copy(deep=True)
salaries_copy_1.head()

salaries_copy_1.drop(['salary'], inplace=True,axis=1)
print(salaries_copy_1.head())

test_copy_1 = test.copy(deep=True)
print(test_copy_1.head())

   work_year experience_level employment_type        job_title  \
0       2022               SE              FT    Data Engineer   
1       2022               SE              FT    Data Engineer   
2       2021               MI              FT  BI Data Analyst   
3       2021               MI              CT      ML Engineer   
4       2021               MI              FT    Data Engineer   

  salary_currency  salary_in_usd employee_residence  remote_ratio  \
0             USD         140250                 US           100   
1             USD         135000                 US           100   
2             USD         100000                 US           100   
3             USD         270000                 US           100   
4             EUR          26005                 RO             0   

  company_location company_size  
0               US            M  
1               US            M  
2               US            M  
3               US            L  
4               US

In [7]:
test_copy_1.shape, salaries_copy_1.shape

((107, 9), (500, 10))

In [8]:
test_copy_1['y'] = -1
test_copy_1.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y
0,2020,SE,FT,Machine Learning Scientist,USD,JP,0,JP,S,-1
1,2020,MI,FT,Lead Data Analyst,USD,US,100,US,L,-1
2,2020,MI,FT,Data Analyst,USD,US,100,US,L,-1
3,2020,MI,FT,Machine Learning Engineer,CNY,CN,0,CN,M,-1
4,2020,MI,FT,Product Data Analyst,INR,IN,100,IN,L,-1


In [9]:
test_copy_1.shape, salaries_copy_1.shape

((107, 10), (500, 10))

In [10]:
todo = pd.concat([test_copy_1, salaries_copy_1])

In [11]:
todo.job_title = todo.job_title.apply(lambda x: dictio_job_title[x])
todo.employment_type = todo.employment_type.apply(lambda x: dictio_employment_type[x])
todo.employee_residence = todo.employee_residence.apply(lambda x: dictio_employee_residence[x])
todo.experience_level = todo.experience_level.apply(lambda x: dictio_experience_level[x])
todo.company_size = todo.company_size.apply(lambda x: dictio_company_size[x])
todo.company_location = todo.company_location.apply(lambda x: dictio_company_location[x])
todo.salary_currency = todo.salary_currency.apply(lambda x: dictio_salary_currency[x])
todo.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,1,2,13,3,2,0,3,2,-1.00,NaN
1,2020,3,2,9,3,44,100,40,0,-1.00,NaN
2,2020,3,2,31,3,44,100,40,0,-1.00,NaN
3,2020,3,2,16,2,41,0,35,1,-1.00,NaN
4,2020,3,2,29,11,43,100,39,0,-1.00,NaN


In [12]:
ok = todo.loc[todo['y'] == -1]
ok.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,1,2,13,3,2,0,3,2,-1.00,NaN
1,2020,3,2,9,3,44,100,40,0,-1.00,NaN
2,2020,3,2,31,3,44,100,40,0,-1.00,NaN
3,2020,3,2,16,2,41,0,35,1,-1.00,NaN
4,2020,3,2,29,11,43,100,39,0,-1.00,NaN


In [13]:
ok.shape

(107, 11)

In [14]:
ok_s = todo.loc[todo['y'] != -1]
ok_s.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2022,1,2,39,3,44,100,40,1,NaN,140250.00
1,2022,1,2,39,3,44,100,40,1,NaN,135000.00
2,2021,3,2,8,3,44,100,40,1,NaN,100000.00
3,2021,3,0,36,3,44,100,40,0,NaN,270000.00
4,2021,3,2,39,12,37,0,40,0,NaN,26005.00


In [15]:
ok_s.shape

(500, 11)

In [16]:
ok_s.drop('y', axis=1, inplace=True)
ok.drop('salary_in_usd', axis=1, inplace=True)
ok.drop('y', axis=1, inplace=True)

## Modelos

In [17]:
X = ok_s.drop('salary_in_usd', axis = 1)
y = ok_s.salary_in_usd

In [18]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((400, 9), (100, 9), (400,), (100,))

In [19]:
lc = Lc(predictions=True)
models, predictions = lc.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.37it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.04,0.03,None,0.05,0.01
RidgeClassifierCV,0.04,0.03,None,0.01,0.01
RidgeClassifier,0.04,0.03,None,0.01,0.02
LinearDiscriminantAnalysis,0.03,0.02,None,0.04,0.01
GaussianNB,0.03,0.02,None,0.04,0.02
RandomForestClassifier,0.02,0.02,None,0.02,0.19
PassiveAggressiveClassifier,0.03,0.02,None,0.02,0.11
LinearSVC,0.02,0.02,None,0.02,0.42
LabelSpreading,0.02,0.01,None,0.02,0.03


In [24]:
lr = Lr(predictions=True)
models, predictions = lr.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 23.05it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
HistGradientBoostingRegressor,0.31,0.37,49566.89,0.19
LGBMRegressor,0.30,0.36,49993.54,0.04
BaggingRegressor,0.25,0.32,51512.29,0.02
GradientBoostingRegressor,0.24,0.31,52129.13,0.04
RandomForestRegressor,0.22,0.29,52718.03,0.14
OrthogonalMatchingPursuit,0.09,0.18,56815.32,0.01
RANSACRegressor,0.09,0.17,57025.78,0.04
PassiveAggressiveRegressor,0.07,0.15,57553.61,0.03
GammaRegressor,0.07,0.15,57591.34,0.01


In [25]:
#Inicializamos modelos
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
dtr=DTR()
lgbmr=LGBMR()
xgbrf = XGBRF()
linreg = LinReg()
knr = KNR()
logreg = LogReg()
hgbr = HGBR()
gr = GR()

#Entrenamientos
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train, verbose=0)
dtr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)
xgbrf.fit(X_train, y_train)
linreg.fit(X_train, y_train)
knr.fit(X_train, y_train)
logreg.fit(X_train, y_train)
hgbr.fit(X_train, y_train)
gr.fit(X_train, y_train)

#Predicciones
y_pred_rfr = rfr.predict(X_test)
y_pred_etr = etr.predict(X_test)
y_pred_gbr = gbr.predict(X_test)
y_pred_xgbr = xgbr.predict(X_test)
y_pred_ctr = ctr.predict(X_test)
y_pred_dtr = dtr.predict(X_test)
y_pred_lgbmr = lgbmr.predict(X_test)
y_pred_xgbrf = xgbrf.predict(X_test)
y_pred_linreg = linreg.predict(X_test)
y_pred_knr = knr.predict(X_test)
y_pred_logreg = logreg.predict(X_test)
y_pred_hgbr = hgbr.predict(X_test)
y_pred_gr = gr.predict(X_test)

#Calculo de error
print(mse(y_test, y_pred_rfr, squared=False), mse(y_test, y_pred_etr, squared=False), mse(y_test, y_pred_gbr, squared=False), 
mse(y_test, y_pred_xgbr, squared=False), mse(y_test, y_pred_ctr, squared=False), mse(y_test, y_pred_dtr, squared=False),
mse(y_test, y_pred_lgbmr, squared=False), mse(y_test, y_pred_xgbrf, squared=False), mse(y_test, y_pred_linreg, squared=False),
mse(y_test, y_pred_knr, squared=False), mse(y_test, y_pred_logreg, squared=False), mse(y_test, y_pred_hgbr, squared=False),
mse(y_test, y_pred_gr, squared=False))

52822.19189450995 68674.86048415437 52165.03797728136 58999.391441863736 53659.00017707421 68965.83020502688 50242.26721440652 56412.025822013915 60078.4364277169 57909.382264900734 60101.66811886339 49597.445495721564 55270.28890807532


### Mejor Modelo: HistGradientBoostingRegressor

In [26]:
hgbr.fit(X_train, y_train)

y_pred_hgbr = hgbr.predict(ok)
y_pred_hgbr[:10]

len(y_pred_hgbr)

107

## Exportar

In [27]:
res = pd.read_csv('../data/muestra.csv')
res.salary_in_usd = y_pred_hgbr
res.head()

,id,salary_in_usd
0,0,90685.78
1,1,212913.45
2,2,140019.04
3,3,75957.92
4,4,33838.80


In [28]:
res.to_csv('../data/resultado_hgbr_contodo_curren.csv', index = False)